# Convert  Object Store datapoints - Airbus NationWide - into Visual search Map

In [1]:
import pandas
print(pandas.__version__)

0.24.2


In [2]:
import folium
print(folium.__version__)

0.14.0


In [3]:
import numpy as np 
import pandas as pd 
import folium
from folium.plugins import Search
from functools import reduce
import math
# No need since 28Apr23 - Lat/Lon provided by Airbus
# Convert BNG to LAT/LON
#from bng_latlon import OSGB36toWGS84
#
input_coord_csv_file='data/output/NBS_Portfolio_Bristol_latlon_230216_csv/part-00000-5c3ba62f-e549-448a-bfa0-6d70538207c5-c000.csv'
df_nationwide_loanbook = pd.read_csv(input_coord_csv_file, header = 0 )


In [4]:
df_nationwide_loanbook.describe()

,ID,UPRN,POSTCODE_FLAG,EASTING,NORTHING,RHIGHSCORE,RHIGHSCORE_ADJ,CHIGHSCORE,CHIGHSCORE_ADJ,SWHIGHSCORE,...,RCP45_2050S_DEF_C_RR,RCP85_2050S_UNDEF_F_RR,RCP85_2050S_UNDEF_P_RR,RCP85_2050S_UNDEF_T_RR,RCP85_2050S_UNDEF_C_RR,RCP85_2050S_DEF_F_RR,RCP85_2050S_DEF_T_RR,RCP85_2050S_DEF_C_RR,LATITUDE,LONGITUDE
count,18085.000000,1.808500e+04,18085.000000,18085.000000,18085.000000,18085.000000,18085.00000,18085.000000,18085.00000,18085.000000,...,18075.000000,18075.000000,18075.000000,18075.000000,18075.000000,18075.000000,18075.000000,18075.000000,18085.000000,18085.000000
mean,9043.000000,8.901323e+09,0.002765,363001.261979,174118.554777,0.517556,0.44031,0.370805,0.09953,0.551562,...,4.389986,1.399336,3.404869,1.235076,5.336985,1.399336,0.891286,5.030152,51.464792,-2.534008
std,5220.834145,2.814950e+10,0.052509,5482.263478,5092.701827,2.950690,2.63575,2.569122,1.14896,1.643107,...,15.618402,10.035297,13.124174,10.103181,18.234758,10.035297,7.390518,17.074913,0.045734,0.078986
min,1.000000,1.470000e+02,0.000000,350002.000000,160034.000000,0.000000,0.00000,0.000000,0.00000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,51.338307,-2.721267
25%,4522.000000,1.116180e+05,0.000000,359164.950000,170934.000000,0.000000,0.00000,0.000000,0.00000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,51.436011,-2.589203
50%,9043.000000,3.731460e+05,0.000000,361955.000000,174209.250000,0.000000,0.00000,0.000000,0.00000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,51.465525,-2.549161
75%,13564.000000,5.924060e+05,0.000000,365945.000000,177516.230000,0.000000,0.00000,0.000000,0.00000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,51.495229,-2.491613
max,18085.000000,2.000029e+11,1.000000,379995.000000,186975.000000,22.000000,22.00000,20.000000,20.00000,10.000000,...,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,51.581107,-2.288625


In [5]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(df_nationwide_loanbook.dtypes)



ID                                    int64
UPRN                                  int64
POSTCODE_FLAG                         int64
EASTING                             float64
NORTHING                            float64
RHIGHSCORE                            int64
RHIGHSCORE_ADJ                        int64
CHIGHSCORE                            int64
CHIGHSCORE_ADJ                        int64
SWHIGHSCORE                           int64
SWHIGHSCORE_ADJ                       int64
GWHIGHSCORE                           int64
COMBINEDSCORE                         int64
COMBINEDSCORE_ADJ                     int64
R_RCP26_2050_UD                     float64
R_RCP26_2050_D                      float64
S_RCP26_2050_UD                     float64
S_RCP26_2050_D                      float64
C_RCP26_2050_UD                     float64
C_RCP26_2050_D                      float64
COMB_RCP26_2050_UD                  float64
COMB_RCP26_2050_D                   float64
R_RCP45_2050_UD                 

In [6]:
df_nationwide_loanbook.dtypes

ID                          int64
UPRN                        int64
POSTCODE_FLAG               int64
EASTING                   float64
NORTHING                  float64
RHIGHSCORE                  int64
RHIGHSCORE_ADJ              int64
CHIGHSCORE                  int64
CHIGHSCORE_ADJ              int64
SWHIGHSCORE                 int64
SWHIGHSCORE_ADJ             int64
GWHIGHSCORE                 int64
COMBINEDSCORE               int64
COMBINEDSCORE_ADJ           int64
R_RCP26_2050_UD           float64
R_RCP26_2050_D            float64
S_RCP26_2050_UD           float64
S_RCP26_2050_D            float64
C_RCP26_2050_UD           float64
C_RCP26_2050_D            float64
COMB_RCP26_2050_UD        float64
COMB_RCP26_2050_D         float64
R_RCP45_2050_UD           float64
R_RCP45_2050_D            float64
S_RCP45_2050_UD           float64
S_RCP45_2050_D            float64
C_RCP45_2050_UD           float64
C_RCP45_2050_D            float64
COMB_RCP45_2050_UD        float64
COMB_RCP45_205

In [7]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(df_nationwide_loanbook.head(12))

    ID  UPRN  POSTCODE_FLAG    EASTING   NORTHING  RHIGHSCORE  RHIGHSCORE_ADJ  \
0    1   147              0  358754.72  179718.88           0               0   
1    2   148              0  358946.00  179716.00           0               0   
2    7   155              0  358974.00  179709.00           0               0   
3    8   157              0  358763.00  179704.00           0               0   
4    9   172              0  358792.00  179695.00           0               0   
5    3   180              0  356788.00  179691.00          18               5   
6    4   183              0  356598.00  179689.00           0               0   
7    5   186              0  358874.00  179687.00           0               0   
8    6   198              0  358725.00  179680.00           0               0   
9   10   223              0  358888.00  179670.00           0               0   
10  11   233              0  358974.00  179667.00           0               0   
11  12   292              0 

In [8]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(df_nationwide_loanbook.tail(20))

          ID          UPRN  POSTCODE_FLAG    EASTING   NORTHING  RHIGHSCORE  \
18065  18056  100121180318              0  357488.00  162920.00          18   
18066  18057  100121180369              0  357503.00  162915.00          20   
18067  18044  100121180376              0  357603.18  162866.30           1   
18068  18058  100121180399              0  357682.00  163408.00           0   
18069  18050  100121180470              0  356297.00  161612.00           0   
18070  18071  100121396620              0  376050.00  166305.00           0   
18071  18072  100121396679              0  374926.00  165642.00           0   
18072  18073  100121397216              0  375612.00  165184.00           0   
18073  18080  200001299301              0  379793.00  160537.00           0   
18074  18078  200001299431              0  378215.00  161733.00           0   
18075  18079  200001300211              0  379865.00  160843.00           0   
18076  18074  200001300253              0  379842.00

In [9]:
# display all distinct CURRENT_ENERGY_RATING values
column_list = ['CURRENT_ENERGY_RATING']
for column_name in column_list:
      print(df_nationwide_loanbook[column_name].unique())

['E' 'D' nan 'C' 'B' 'F' 'G' 'A']


In [10]:
# display all distinct COMBINEDSCORE_ADJ values
column_list = ['COMBINEDSCORE_ADJ']
for column_name in column_list:
      print(df_nationwide_loanbook[column_name].unique())

[ 0  2  5  8  1  3 23  7 10  6 22 11  9  4 12 26 28 17 18 21 15 30 20 24
 27 25 19 34 37 13 14 16 32 42 35]


In [11]:
# display all distinct COMBINEDSCORE values
column_list = ['COMBINEDSCORE']
for column_name in column_list:
      print(df_nationwide_loanbook[column_name].unique())

[ 0  2 18  5  8  1  3 23  7 21 20 12 17 25 22  4 10 28 16 14  9  6 26 24
 15 30 27 19 35 42 45 13 33 48 31 38 40 37 29 11 39 43 34 32 36 44 47]


In [12]:
# CONTINUE WITH MAP LeafLeat Folium map generation of HTML

In [13]:
# Increase display in Jupyter
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [14]:
df_nationwide_loanbook.shape

(18085, 200)

In [15]:
lat= 51.44291
lon= -2.57643

In [16]:
#Color codes for EPC rating scale
colors_epc={
    np.nan:'#deebf6',
    'A':'#00a54f',
    'B':'#4cb848',
    'C':'#bed630',
    'D':'#fff101',
    'E':'#fcb814',
    'F':'#f36e21',
    'G':'#ee1d23',
}

In [17]:
#Color codes for flood rating scale
colours_flood={
    'no-colour':'#deebf6',
    'green':'#a5d45d',
    'amber':'#ecc63a',
    'red':'#d92424',
    'black-1':'#919191',
    'black-2':'#242424',
    'purple':'#9370db',
}

# Function to convert number into string
# Switcher is dictionary data type here
### Planned for COMBINEDSCORE/COMBINEDSCORE_ADJ -  Combined Floodability undefended/defended - score
def FloodRatingSwitch(undefend_sum,defend_sum):
    value_search = undefend_sum if (undefend_sum > defend_sum) else defend_sum
    # print('value_search_='+str(value_search))
    list_no_colour = [0]
    list_green = [1,2,3,4,5,6,7,8,9,10]
    list_amber = [11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29]
    list_red = [30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45]
    list_black1 = [45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60]
    list_black2 = [61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100]
    # Evaluate from List
    if value_search in list_no_colour:
        return 'no-colour'
    elif value_search in list_green:
        return 'green'
    elif value_search in list_amber:
        return 'amber'
    elif value_search in list_red:
        return 'red'
    elif value_search in list_black1:
        return 'black-1'
    elif value_search in list_black2:
        return 'black-2'
    else:
        return 'purple' #ERROR out-of-bounds
    
    # get() method of dictionary data type returns
    # value of passed argument if it is present
    # in dictionary otherwise second argument will
    # be assigned as default value of passed argument
    # return switcher.get(value_search, 'purple') # 'purple':'#9370db', ERROR

In [18]:
# Creating the popup labels for energy rating data 

def popup_html_epc(row):
    i = row
    uprn_id= df_nationwide_loanbook['UPRN'].iloc[i] 
    address= 'Property_type='+str(df_nationwide_loanbook['PROPERTY_TYPE'].iloc[i])+'. '+str(df_nationwide_loanbook['BUILT_FORM'].iloc[i])+'. '+  str( df_nationwide_loanbook['POSTCODE_FLAG'].iloc[i] ) +'. Easting/Norting='+ str( df_nationwide_loanbook['EASTING'].iloc[i] ) +'/'+ str( df_nationwide_loanbook['NORTHING'].iloc[i] ) +'. Latitude/Longitude='+ str( df_nationwide_loanbook['LATITUDE'].iloc[i] ) +'/'+ str( df_nationwide_loanbook['LONGITUDE'].iloc[i] )
    Epc_Rating = df_nationwide_loanbook['CURRENT_ENERGY_RATING'].iloc[i]
    Epc_Energy_Cons_Kwh = df_nationwide_loanbook['ENERGY_CONSUMPTION_CURRENT'].iloc[i]
    Co2_Emission = df_nationwide_loanbook['CO2_EMISSIONS_CURRENT'].iloc[i]

    left_col_color = "#808080"
    right_col_color = "#dcdcdc"
    
    html = """
<!DOCTYPE html>
<html>
<center><h4 style="margin-bottom:5"; width="200px">UPRN ID:{}</h4>""".format(uprn_id) + """</center>
<center> <table style="height: 126px; width: 305px;">
<tbody>
<tr>
<td style="background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Property </span></td>
<td style="width: 150px;background-color: """+ right_col_color +""";">"""+ address + """</td>
</tr>
<tr>
<td style="background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Epc Rating </span></td>
<td style="width: 150px;background-color: """+ right_col_color +""";">{}</td>""".format(Epc_Rating) + """
</tr>
<tr>
<td style="background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Epc Energy Consumption Kwh </span></td>
<td style="width: 150px;background-color: """+ right_col_color +""";">{} kWh/m\u00b2/year</td>""".format(Epc_Energy_Cons_Kwh) + """
</tr>
<tr>
<td style="background-color: """+ left_col_color +""";"><span style="color: #ffffff;">CO\u2082 Emission Current</span></td>
<td style="width: 150px;background-color: """+ right_col_color +""";">{}</td>""".format(Co2_Emission) + """
</tr>
</tbody>
</table></center>

<h5 style="margin-bottom:5px; margin-left:5px; font-weight: bold"> EPC Energy Rating and Consumption</h5>
<center> <table style="height: 50px; width: 600px;">
<thead>
<tr>
<th style= "border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;"></span></th>
<th style= "border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;"> 2023y Current </span></th>
<th style= "border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;"> 2023y Potential </span></th>
</tr>
</thead>
<tbody>
<tr>
<td style= "border: 1px solid white; width=100px; background-color: """+ left_col_color +""";"><span style="color: #ffffff;"> EPC Energy Rating and Consumption </span></td>
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">{} ({} kWh/m\u00b2/year)</td>""".format(df_nationwide_loanbook['CURRENT_ENERGY_RATING'].iloc[i],df_nationwide_loanbook['ENERGY_CONSUMPTION_CURRENT'].iloc[i]) + """
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">{} ({} kWh/m\u00b2/year)</td>""".format(df_nationwide_loanbook['POTENTIAL_ENERGY_RATING'].iloc[i],df_nationwide_loanbook['ENERGY_CONSUMPTION_POTENTIAL'].iloc[i]) + """
</tr>
</tbody>
</table> </center>

<h5 style="margin-bottom:5px; margin-left:5px; font-weight: bold"> Property CO\u2082 Emissions</h5>
<center> <table style="height: 50px; width: 600px;">
<thead>
<tr>
<th style= "border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;"></span></th>
<th style= "border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;"> 2023y Current </span></th>
<th style= "border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;"> 2023y Potential </span></th>
</tr>
</thead>
<tbody>
<tr>
<td style= "border: 1px solid white; width=100px; background-color: """+ left_col_color +""";"><span style="color: #ffffff;"> CO\u2082 Emissions </span></td>
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">{} CO\u2082 Tonnes/year </td>""".format(df_nationwide_loanbook['CO2_EMISSIONS_CURRENT'].iloc[i]) + """
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">{} CO\u2082 Tonnes/year </td>""".format(df_nationwide_loanbook['CO2_EMISSIONS_POTENTIAL'].iloc[i]) + """
</tr>
</tbody>
</table> </center>

<h5 style="margin-bottom:5px; margin-left:5px; font-weight: bold"> Energy Efficiency</h5>
<center> <table style="height: 50px; width: 600px;">
<thead>
<tr>
<th style= "border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;"></span></th>
<th style= "border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;"> 2023y Current </span></th>
<th style= "border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;"> 2023y Potential </span></th>
</tr>
</thead>
<tbody>
<tr>
<td style= "border: 1px solid white; width=100px; background-color: """+ left_col_color +""";"><span style="color: #ffffff;"> Energy Efficiency </span></td>
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">{}% Based on cost of energy, i.e. heating, water and lighting [in kWh/year] mult. by fuel costs. (£/m²/year cost is derived from kWh). </td>""".format(df_nationwide_loanbook['CURRENT_ENERGY_EFFICIENCY'].iloc[i]) + """
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">{}% Based on cost of energy, i.e. heating, water and lighting [in kWh/year] mult. by fuel costs. (£/m²/year cost is derived from kWh). </td>""".format(df_nationwide_loanbook['POTENTIAL_ENERGY_EFFICIENCY'].iloc[i]) + """
</tr>
</tbody>
</table> </center>

</html>
"""
    return html

In [19]:
# Creating the popup labels for flood rating data

def popup_html_flood(row):
    i = row
    uprn_id=df_nationwide_loanbook['UPRN'].iloc[i] 
    address= 'Property_type='+str(df_nationwide_loanbook['PROPERTY_TYPE'].iloc[i])+'. '+str(df_nationwide_loanbook['BUILT_FORM'].iloc[i])+'. '+  str( df_nationwide_loanbook['POSTCODE_FLAG'].iloc[i] ) +'. Easting/Norting='+ str( df_nationwide_loanbook['EASTING'].iloc[i] ) +'/'+ str( df_nationwide_loanbook['NORTHING'].iloc[i] ) +'. Latitude/Longitude='+ str( df_nationwide_loanbook['LATITUDE'].iloc[i] ) +'/'+ str( df_nationwide_loanbook['LONGITUDE'].iloc[i] )
    river_flooding_first = df_nationwide_loanbook['RHIGHSCORE'].iloc[i]
    coastal_flooding_first = df_nationwide_loanbook['CHIGHSCORE'].iloc[i]
    surf_water_flooding_first = df_nationwide_loanbook['SWHIGHSCORE'].iloc[i]
    combined_scoring_flooding_first = df_nationwide_loanbook['COMBINEDSCORE'].iloc[i]
    
    left_col_color = "#808080"
    right_col_color = "#dcdcdc"
    
    html = """
<!DOCTYPE html>
<html>
<center><h4 style="margin-bottom:5"; width="200px">UPRN ID:{}</h4>""".format(uprn_id) + """</center>
<center> <table style="height: 90px; width: 500px;">
<tbody>
<tr>
<td style= "border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Property </span></td>
<td style="border: 1px solid white; width: 150px;background-color: """+ right_col_color +""";">"""+ address + """</td>
</tr>
<tr>
<td style="border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Highest score from the Defended river scores  </span></td>
<td style="border: 1px solid white; width: 150px;background-color: """+ right_col_color +""";">{}</td>""".format(river_flooding_first) + """
</tr>
<tr>
<td style="border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Highest score from the Defended coastal scores </span></td>
<td style="border: 1px solid white; width: 150px;background-color: """+ right_col_color +""";">{}</td>""".format(coastal_flooding_first) + """
</tr>
<tr>
<td style="border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;"> Highest score from the Defended surface water scores </span></td>
<td style="border: 1px solid white; width: 150px;background-color: """+ right_col_color +""";">{}</td>""".format(surf_water_flooding_first) + """
</tr>
<tr>
<td style="border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Combined Sum of all Scores - Defended River, Coastal and Surface Water</span></td>
<td style="border: 1px solid white; width: 150px;background-color: """+ right_col_color +""";">{}</td>""".format(combined_scoring_flooding_first) + """
</tr>
</tbody>
</table>
</center>

<h5 style="margin-bottom:5px; margin-left:5px; font-weight: bold">River</h5>
<center> <table style="height: 50px; width: 600px;">
<thead>
<tr>
<th style= "border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;"></span></th>
<th style= "border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;"> 2023y defended </span></th>
<th style= "border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;"> 2023y undefended </span></th>
</tr>
</thead>
<tbody>
<tr>
<td style= "border: 1px solid white; width=100px; background-color: """+ left_col_color +""";"><span style="color: #ffffff;"> Highest score from the undefended/defended river scores </span></td>
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">{}</td>""".format(df_nationwide_loanbook['RHIGHSCORE'].iloc[i]) + """
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">{}</td>""".format(df_nationwide_loanbook['RHIGHSCORE_ADJ'].iloc[i]) + """
</tr>
</tbody>
</table> </center>

<h5 style="margin-bottom:5px; margin-left:5px; font-weight: bold">Coastal</h5>
<center> <table style="height: 50px; width: 600px;">
<thead>
<tr>
<th style= "border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;"></span></th>
<th style= "border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;"> 2023y defended </span></th>
<th style= "border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;"> 2023y undefended </span></th>
</tr>
</thead>
<tbody>
<tr>
<td style="border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;"> Highest score from the undefended/defended coastal scores </span></td>
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">{}</td>""".format(df_nationwide_loanbook['CHIGHSCORE'].iloc[i]) + """
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">{}</td>""".format(df_nationwide_loanbook['CHIGHSCORE_ADJ'].iloc[i]) + """
</tr>
</tbody>
</table> </center>

<h5 style="margin-bottom:5px; margin-left:5px; font-weight: bold">Surface Water</h5>
<center> <table style="height: 50px; width: 600px;">
<thead>
<tr>
<th style= "border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;"></span></th>
<th style= "border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;"> 2023y defended </span></th>
<th style= "border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;"> 2023y undefended </span></th>
</tr>
</thead>
<tbody>
<tr>
<td style="border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;"> Highest score from the undefended/defended surface water scores </span></td>
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">{}</td>""".format(df_nationwide_loanbook['SWHIGHSCORE'].iloc[i]) + """
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">{}</td>""".format(df_nationwide_loanbook['SWHIGHSCORE_ADJ'].iloc[i]) + """
</tr>
</tbody>
</table> </center>

<h5 style="margin-bottom:5px; margin-left:5px; font-weight: bold">Ground Water</h5>
<center> <table style="height: 50px; width: 600px;">
<thead>
<tr>
<th style= "border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;"></span></th>
<th style= "border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;"> 2023y highest </span></th>
</tr>
</thead>
<tbody>
<tr>
<td style="border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;"> Highest score from the ground water scores </span></td>
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">{}</td>""".format(df_nationwide_loanbook['GWHIGHSCORE'].iloc[i]) + """
</tr>
</tbody>
</table> </center>

<h5 style="margin-bottom:5px; margin-left:5px; font-weight: bold">All Scores</h5>
<center> <table style="height: 50px; width: 600px;">
<thead>
<tr>
<th style= "border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;"></span></th>
<th style= "border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;"> 2023y defended </span></th>
<th style= "border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;"> 2023y undefended </span></th>
</tr>
</thead>
<tbody>
<tr>
<td style="border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;"> The sum of all the undefended/defended high scores </span></td>
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">{}</td>""".format(df_nationwide_loanbook['COMBINEDSCORE'].iloc[i]) + """
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">{}</td>""".format(df_nationwide_loanbook['COMBINEDSCORE_ADJ'].iloc[i]) + """
</tr>
</tbody>
</table> </center>

<h5 style="margin-bottom:5px; margin-left:5px; font-weight: bold">Loan Details</h5>
<center> <table style="height: 50px; width:600px;">
<thead>
<tr>
<th style= "border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;">UPRN </span></th>
<th style= "border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Current Loan Balance</span></th>
<th style= "border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Current Loan to Value LTV</span></th>
</tr>
</thead>
<tbody>
<tr>
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">{}</td>""".format(df_nationwide_loanbook['UPRN'].iloc[i]) + """
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">£{}</td>""".format(df_nationwide_loanbook['CURRENTLOAN'].iloc[i]) + """
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">%{}</td>""".format(df_nationwide_loanbook['CURRENTLTV'].iloc[i]) + """
</tr>
</tbody>
</table> </center>
</html>
""" 
    return html

In [20]:
# Creating the popup labels for Airbus/NWide  loan data

def popup_html_index(row):
    i = row
    uprn_id=df_nationwide_loanbook['UPRN'].iloc[i] 
    address= 'Property_type='+str(df_nationwide_loanbook['PROPERTY_TYPE'].iloc[i])+'. '+str(df_nationwide_loanbook['BUILT_FORM'].iloc[i])+'. '+  str( df_nationwide_loanbook['POSTCODE_FLAG'].iloc[i] ) +'. Easting/Norting='+ str( df_nationwide_loanbook['EASTING'].iloc[i] ) +'/'+ str( df_nationwide_loanbook['NORTHING'].iloc[i] ) +'. Latitude/Longitude='+ str( df_nationwide_loanbook['LATITUDE'].iloc[i] ) +'/'+ str( df_nationwide_loanbook['LONGITUDE'].iloc[i] )
    river_flooding_first = df_nationwide_loanbook['RHIGHSCORE'].iloc[i]
    coastal_flooding_first = df_nationwide_loanbook['RHIGHSCORE'].iloc[i] 
    surf_water_flooding_first = df_nationwide_loanbook['RHIGHSCORE'].iloc[i]
    
    left_col_color = "#808080"
    right_col_color = "#dcdcdc"
    
    html = """
<!DOCTYPE html>
<html>
<center><h4 style="margin-bottom:5"; width="200px"> UPRN :{}</h4>""".format(uprn_id) + """</center>
<center> <table style="height: 90px; width: 600px;">
<tbody>
<tr>
<td style= "border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Property </span></td>
<td style="border: 1px solid white; width: 150px;background-color: """+ right_col_color +""";">"""+ address + """</td>
</tr>
<tr>
<td style="border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Current EPC Energy Rating and Consumption</span></td>
<td style="border: 1px solid white; width: 150px;background-color: """+ right_col_color +""";">{} ({} kWh/m\u00b2/year)</td>""".format(df_nationwide_loanbook['CURRENT_ENERGY_RATING'].iloc[i],df_nationwide_loanbook['ENERGY_CONSUMPTION_CURRENT'].iloc[i]) + """
</tr>
<tr>
<td style="border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Potential EPC Energy Rating and Consumption</span></td>
<td style="border: 1px solid white; width: 150px;background-color: """+ right_col_color +""";">{} ({} kWh/m\u00b2/year)</td>""".format(df_nationwide_loanbook['POTENTIAL_ENERGY_RATING'].iloc[i],df_nationwide_loanbook['ENERGY_CONSUMPTION_POTENTIAL'].iloc[i]) + """
</tr>
<tr>
<td style="border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Current CO\u2082 Emissions</span></td>
<td style="border: 1px solid white; width: 150px;background-color: """+ right_col_color +""";">{} CO\u2082 Tonnes/year </td>""".format(df_nationwide_loanbook['CO2_EMISSIONS_CURRENT'].iloc[i]) + """
</tr>
<tr>
<td style="border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Potential CO\u2082 Emissions</span></td>
<td style="border: 1px solid white; width: 150px;background-color: """+ right_col_color +""";">{} CO\u2082 Tonnes/year </td>""".format(df_nationwide_loanbook['CO2_EMISSIONS_POTENTIAL'].iloc[i]) + """
</tr>
<tr>
<td style="border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Current Energy Efficiency</span></td>
<td style="border: 1px solid white; width: 150px;background-color: """+ right_col_color +""";">{}% Based on cost of energy, i.e. heating, water and lighting [in kWh/year] mult. by fuel costs. (£/m²/year cost is derived from kWh). </td>""".format(df_nationwide_loanbook['CURRENT_ENERGY_EFFICIENCY'].iloc[i]) + """
</tr>
<tr>
<td style="border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Potential Energy Efficiency</span></td>
<td style="border: 1px solid white; width: 150px;background-color: """+ right_col_color +""";">{}% Based on cost of energy, i.e. heating, water and lighting [in kWh/year] mult. by fuel costs. (£/m²/year cost is derived from kWh). </td>""".format(df_nationwide_loanbook['POTENTIAL_ENERGY_EFFICIENCY'].iloc[i]) + """
</tr>
</tbody>
</table>
</center>

<h5 style="margin-bottom:5px; margin-left:5px; font-weight: bold">River</h5>
<center> <table style="height: 50px; width: 600px;">
<thead>
<tr>
<th style= "border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;"></span></th>
<th style= "border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;"> 2023y defended </span></th>
<th style= "border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;"> 2023y undefended </span></th>
</tr>
</thead>
<tbody>
<tr>
<td style= "border: 1px solid white; width=100px; background-color: """+ left_col_color +""";"><span style="color: #ffffff;"> Highest score from the undefended/defended river scores </span></td>
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">{}</td>""".format(df_nationwide_loanbook['RHIGHSCORE'].iloc[i]) + """
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">{}</td>""".format(df_nationwide_loanbook['RHIGHSCORE_ADJ'].iloc[i]) + """
</tr>
</tbody>
</table> </center>

<h5 style="margin-bottom:5px; margin-left:5px; font-weight: bold">Coastal</h5>
<center> <table style="height: 50px; width: 600px;">
<thead>
<tr>
<th style= "border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;"></span></th>
<th style= "border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;"> 2023y defended </span></th>
<th style= "border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;"> 2023y undefended </span></th>
</tr>
</thead>
<tbody>
<tr>
<td style="border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;"> Highest score from the undefended/defended coastal scores </span></td>
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">{}</td>""".format(df_nationwide_loanbook['CHIGHSCORE'].iloc[i]) + """
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">{}</td>""".format(df_nationwide_loanbook['CHIGHSCORE_ADJ'].iloc[i]) + """
</tr>
</tbody>
</table> </center>

<h5 style="margin-bottom:5px; margin-left:5px; font-weight: bold">Surface Water</h5>
<center> <table style="height: 50px; width: 600px;">
<thead>
<tr>
<th style= "border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;"></span></th>
<th style= "border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;"> 2023y defended </span></th>
<th style= "border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;"> 2023y undefended </span></th>
</tr>
</thead>
<tbody>
<tr>
<td style="border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;"> Highest score from the undefended/defended surface water scores </span></td>
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">{}</td>""".format(df_nationwide_loanbook['SWHIGHSCORE'].iloc[i]) + """
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">{}</td>""".format(df_nationwide_loanbook['SWHIGHSCORE_ADJ'].iloc[i]) + """
</tr>
</tbody>
</table> </center>

<h5 style="margin-bottom:5px; margin-left:5px; font-weight: bold">Ground Water</h5>
<center> <table style="height: 50px; width: 600px;">
<thead>
<tr>
<th style= "border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;"></span></th>
<th style= "border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;"> 2023y highest </span></th>
</tr>
</thead>
<tbody>
<tr>
<td style="border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;"> Highest score from the ground water scores </span></td>
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">{}</td>""".format(df_nationwide_loanbook['GWHIGHSCORE'].iloc[i]) + """
</tr>
</tbody>
</table> </center>

<h5 style="margin-bottom:5px; margin-left:5px; font-weight: bold">All Scores</h5>
<center> <table style="height: 50px; width: 600px;">
<thead>
<tr>
<th style= "border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;"></span></th>
<th style= "border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;"> 2023y defended </span></th>
<th style= "border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;"> 2023y undefended </span></th>
</tr>
</thead>
<tbody>
<tr>
<td style="border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;"> The sum of all the undefended/defended high scores </span></td>
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">{}</td>""".format(df_nationwide_loanbook['COMBINEDSCORE'].iloc[i]) + """
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">{}</td>""".format(df_nationwide_loanbook['COMBINEDSCORE_ADJ'].iloc[i]) + """
</tr>
</tbody>
</table> </center>

<h5 style="margin-bottom:5px; margin-left:5px; font-weight: bold">Loan Details</h5>
<center> <table style="height: 50px; width:600px;">
<thead>
<tr>
<th style= "border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;">UPRN </span></th>
<th style= "border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Current Loan Balance</span></th>
<th style= "border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Current Loan to Value LTV</span></th>
</tr>
</thead>
<tbody>
<tr>
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">{}</td>""".format(df_nationwide_loanbook['UPRN'].iloc[i]) + """
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">£{}</td>""".format(df_nationwide_loanbook['CURRENTLOAN'].iloc[i]) + """
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">%{}</td>""".format(df_nationwide_loanbook['CURRENTLTV'].iloc[i]) + """
</tr>
</tbody>
</table> </center>
</html>
""" 
    return html

In [21]:
#Creating lagends

def add_legend(maps, title, colors, labels):
    if len(colors) != len(labels):
        raise ValueError("colors and labels must have the same length.")

    color_label = dict(zip(labels, colors))
    
    legends = ""     
    for label, color in color_label.items():
        legends += f"<li><span style='background:{color}'></span>{label}</li>"
        
    legend_html = f"""
    <div id='legend' class='legend'>
      <div class='legend-title'>{title}</div>
      <div class='legend-scale'>
        <ul class='legend-labels'>
        {legends}
        </ul>
      </div>
    </div>
    """
    script = f"""
        <script type="text/javascript">
        var Layout = (function() {{
                    var flag = false;
                    return function() {{
                        if (!flag) {{
                             var checkExist = setInterval(function() {{
                                       if ((document.getElementsByClassName('leaflet-top leaflet-right').length) || (!flag)) {{
                                          document.getElementsByClassName('leaflet-top leaflet-right')[0].style.display = "flex"
                                          document.getElementsByClassName('leaflet-top leaflet-right')[0].style.flexDirection = "column"
                                          document.getElementsByClassName('leaflet-top leaflet-right')[0].innerHTML += `{legend_html}`;
                                          clearInterval(checkExist);
                                          flag = true;
                                       }}
                                    }}, 100);
                        }}
                    }};
                }})();
        Layout()
        </script>
      """
   

    css = """

    <style type='text/css'>
      .legend {
        z-index:9999;
        float:right;
        background-color: rgb(255, 255, 255);
        border-radius: 5px;
        border: 2px solid #bbb;
        padding: 10px;
        font-size:12px;
        positon: relative;
      }
      .legend .legend-title {
        text-align: left;
        margin-bottom: 5px;
        font-weight: bold;
        font-size: 90%;
        }
      .legend .legend-scale ul {
        margin: 0;
        margin-bottom: 5px;
        padding: 0;
        float: left;
        list-style: none;
        }
      .legend .legend-scale ul li {
        font-size: 80%;
        list-style: none;
        margin-left: 0;
        line-height: 18px;
        margin-bottom: 2px;
        }
      .legend ul.legend-labels li span {
        display: block;
        float: left;
        height: 16px;
        width: 30px;
        margin-right: 5px;
        margin-left: 0;
        border: 0px solid #ccc;
        }
      .legend .legend-source {
        font-size: 80%;
        color: #777;
        clear: both;
        }
      .legend a {
        color: #777;
        }
    </style>
    """

    maps.get_root().header.add_child(folium.Element(script + css))

    return maps

# Display - NationWide & Loanbook  search by index ecad_id - UK Bristol County

In [22]:
#Plotting Nwide_loan data - index search

bristol_airbus_nwide_loanbook=folium.Map(location=[lat,lon],zoom_start=10)
from folium import plugins
cluster=plugins.MarkerCluster().add_to(bristol_airbus_nwide_loanbook)

for d in df_nationwide_loanbook.iterrows(): 
        html = popup_html_index(d[0])
        popup = folium.Popup(folium.Html(html, script=True), max_width=600)
        folium.Marker(location=[d[1]['LATITUDE'], d[1]['LONGITUDE']], popup=popup,name=d[1]["UPRN"]).add_to(cluster)
Search(cluster,search_label='name',placeholder='Search for UPRN ID').add_to(bristol_airbus_nwide_loanbook)            
bristol_airbus_nwide_loanbook.save('data/output/visualization_Bristol_Airbus_Nationwide_loanbook/Bristol_Airbus_Nwide_Loanbook_IndexSearch.html')

# Display Optional
# bristol_airbus_nwide_loanbook

In [23]:
#Plotting EPC_rating data

bristol_epc=folium.Map(location=[lat,lon],zoom_start=10)

for d in df_nationwide_loanbook.iterrows():
        html = popup_html_epc(d[0])
        popup = folium.Popup(folium.Html(html, script=True), max_width=500)
        folium.CircleMarker(
                    [d[1]['LATITUDE'], d[1]['LONGITUDE']],
                    radius=6,
                    color=colors_epc[d[1]["CURRENT_ENERGY_RATING"]],
                    fill=True,
                    fill_color=colors_epc[d[1]["CURRENT_ENERGY_RATING"]],
                    fill_opacity=0.7,
                    popup=popup
            ).add_to(bristol_epc)

bristol_epc = add_legend(bristol_epc, 'Building Energy Rating', colors = list(colors_epc.values()), labels = list(colors_epc.keys()))
bristol_epc.save('data/output/visualization_Bristol_Airbus_Nationwide_loanbook/Bristol_Airbus_Nwide_Energy_Rating.html')

# Display Optional
# bristol_epc

In [24]:
#Plotting flood_rating data

bristol_flood=folium.Map(location=[lat,lon],zoom_start=10)

for d in df_nationwide_loanbook.iterrows():
        html = popup_html_flood(d[0])
        popup = folium.Popup(folium.Html(html, script=True), max_width=500)
        colour_d1point = FloodRatingSwitch(d[1]['COMBINEDSCORE_ADJ'], d[1]['COMBINEDSCORE'])
        folium.CircleMarker(
                    [d[1]['LATITUDE'], d[1]['LONGITUDE']],
                    radius=6,
                    color=colours_flood[colour_d1point],
                    fill=True,
                    fill_color=colours_flood[colour_d1point],
                    fill_opacity=0.7,
                    popup=popup
            ).add_to(bristol_flood)

bristol_flood = add_legend(bristol_flood, 'Likelihood of flooding', colors = list(colours_flood.values()), labels = ('Very Low', 'Low', 'Moderate', 'Moderate to High', 'High', 'Very High', 'ERROR'))
bristol_flood.save('data/output/visualization_Bristol_Airbus_Nationwide_loanbook/Bristol_Airbus_Nwide_Flood_Rating.html')

# Display Optional 
#bristol_flood

In [25]:
exit()

ERROR:root:Invalid alias: The name clear can't be aliased because it is another magic command.
ERROR:root:Invalid alias: The name more can't be aliased because it is another magic command.
ERROR:root:Invalid alias: The name less can't be aliased because it is another magic command.
ERROR:root:Invalid alias: The name man can't be aliased because it is another magic command.
